In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("./word2vec_wrangling.csv")
exercise_to_loop = df["exercise_name"].to_list()

In [3]:
# -*- coding: utf-8 -*-
import re
from konlpy.tag import Mecab, Okt
from collections import Counter
import pandas as pd
import numpy as np

def preprocessing_hangul(text):
    # 개행문자 제거
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result


In [4]:
def yield_combined_df(exercise_name):
    file_name = "#" + exercise_name + "_sum.txt"
    file_1 = "/Users/noopy/FitCuration/" + file_name
    text = open(file_1, 'r', -1, "UTF-8", errors="ignore").read()
    # text

    clean_text = preprocessing_hangul(text)
    clean_text

    # MeCab으로 명사 뽑아내기
    mecab = Mecab()
    noun_list_mecab = mecab.nouns(clean_text)

    # stopwards preprocessing
    stopwords_mecab = ['수','퀄리티','도시','분','전문','스타','년','원',\
                       '월','화','수','목','금','시','앤','일','그램','문'] 

    clean_noun_list_mecab = []

    for n in noun_list_mecab:
        if n not in stopwords_mecab:
            clean_noun_list_mecab.append(n)

    # get top 100 most common nouns
    nouns_mecab = Counter(clean_noun_list_mecab)
    tags_mecab = nouns_mecab.most_common(100)

    # Okt(Twitter 업데이트 버전)으로 서술어 뽑아내기

    twitter = Okt()

    # 4. 각 문장별로 형태소 구분하기
    morphed_list_okt = twitter.pos(clean_text)
    # morphed_list

    # 불용어
    stopwords_Twitter = ["입니다","있는","있습니다","같은","안녕하세요","고마워요","있어요","있게"\
                         ,"있도록","부탁드립니다","하는","합니다","할","하세요","하기","해","됩니다","하여",'잘','된','되고','되어','되었습니다',"없는","드립니다"\
                        ,"되기","하시는","하고","않을","같다","싶다","이런","저런","그런",'바랍니다'\
                        ,"했습니다","했다","해드립니다","하신","하실","않고","해요","가능합니다","하고싶으신"\
                        ,"않으며","주세요","오세요"]

    # 5. 형용사 품사만 뽑아내기
    adj_list_okt = []
    for word, tag in morphed_list_okt:
        if tag in ['Adjective','Verb'] and word not in stopwords_Twitter:
            adj_list_okt.append(word)

    # 6. 선별된 품사별 빈도수 계산 & 상위 빈도 10위 까지 출력
    adj_counts_okt = Counter(adj_list_okt)
    common_adj_okt = adj_counts_okt.most_common(100)
    # common_adj_okt

    noun_df = pd.DataFrame(np.sort(np.array(tags_mecab), axis=1), columns=[exercise_name+"freq",exercise_name+'관련명사'])
    adj_df = pd.DataFrame(np.sort(np.array(common_adj_okt), axis=1), columns=[exercise_name+ "freq",exercise_name+'관련서술어'])
    combined_df = pd.concat([noun_df,adj_df],axis=1)
    return combined_df

In [5]:
exercise_to_loop.sort()
exercise_to_loop[:10]

['PT',
 '검도',
 '기구필라테스',
 '다빈치바디보드',
 '드럼스틱',
 '등산',
 '라틴댄스',
 '매트필라테스',
 '뮤직복싱',
 '바차타']

In [6]:
df0 = yield_combined_df('PT')

for item in exercise_to_loop:
    df0 = pd.concat([df0,yield_combined_df(item)],axis=1)
df0.to_csv("combined_wrangling_db.csv", index=False, header=True)